In [1]:
!pip install pandas
!pip install pyspark
!pip install pyarrow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

24/11/28 00:22:48 WARN Utils: Your hostname, Maazs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.24 instead (on interface en0)
24/11/28 00:22:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 00:22:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

data = [[3, 'Mila', 9, 60301], [12, 'Antonella', None, 31000], [13, 'Emery', None, 67084], [1, 'Kalel', 11, 21241], [9, 'Mikaela', None, 50937], [11, 'Joziah', 6, 28485]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'manager_id', 'salary']).astype({'employee_id':'Int64', 'name':'object', 'manager_id':'Int64', 'salary':'Int64'})

In [4]:
employees = spark.createDataFrame(employees)
employees.show()

+-----------+---------+----------+------+
|employee_id|     name|manager_id|salary|
+-----------+---------+----------+------+
|          3|     Mila|         9| 60301|
|         12|Antonella|      NULL| 31000|
|         13|    Emery|      NULL| 67084|
|          1|    Kalel|        11| 21241|
|          9|  Mikaela|      NULL| 50937|
|         11|   Joziah|         6| 28485|
+-----------+---------+----------+------+



In [10]:
employees \
.where(f"""salary < 30000
       and manager_id is not null
       and manager_id not in {tuple(employees[['employee_id']] \
                              .toPandas()['employee_id'] \
                              .tolist())}
       """) \
.select('employee_id') \
.sort('employee_id') \
.show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+

